In [0]:
from pyspark.sql.functions import col, when

# Read streaming data from the bronze table
bronze_df = spark.readStream.table("desenvolvimento.bronze.artists")

# Replace null values in the 'genres' column with "Not Defined"
silver_df = bronze_df.withColumn("genres", when(col("genres").isNull(), "Not Defined").otherwise(col("genres")))

# Write the transformed data to the silver table
silver_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/desenvolvimento/bronze/checkpoints/artists_to_silver") \
    .trigger(availableNow=True) \
    .table("desenvolvimento.silver.artists")

In [0]:
from pyspark.sql.functions import when, col, to_date, lit

# Read streaming data from the bronze table
bronze_df = spark.readStream.table("desenvolvimento.bronze.tracks")

# Replace null values in the 'genres' column with "Not Defined"
silver_df = (bronze_df.withColumn("key", when(col("key").isNull(), 0).otherwise(col("key")))
    .withColumn("mode", when(col("mode").isNull(), 0).otherwise(col("mode")))
    .withColumn("time_signature", when(col("time_signature").isNull(), 0).otherwise(col("time_signature")))
    .withColumn(
        "release_date",
        when(
            (to_date(col("release_date"), "yyyy-MM-dd") > lit("2025-07-12")) |
            (to_date(col("release_date"), "yyyy-MM-dd") < lit("1900-01-01")) |
            col("release_date").isNull(),
            None
        ).otherwise(col("release_date"))
    ))

# Write the transformed data to the silver table
silver_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/desenvolvimento/bronze/checkpoints/tracks_to_silver") \
    .trigger(availableNow=True) \
    .table("desenvolvimento.silver.tracks")